In [ ]:
# Clean install
!pip install -q ultralytics opencv-python-headless numpy==1.26.4

import os
import sys
import yaml
import shutil
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch

print("✅ Libraries imported")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Patch 1: Fix imread
def patched_imread(filename, flags=cv2.IMREAD_COLOR):
    img = cv2.imread(str(filename), flags)
    if img is not None:
        return img if img.ndim == 3 else img[..., None]
    
    # Fallback
    with open(filename, 'rb') as f:
        arr = np.frombuffer(f.read(), dtype=np.uint8)
    img = cv2.imdecode(arr, flags)
    return img if img is not None and img.ndim == 3 else (img[..., None] if img is not None else None)

import ultralytics.utils.patches
ultralytics.utils.patches.imread = patched_imread
print("✅ imread patched")

# Patch 2: PyTorch 2.6+ compatibility
from ultralytics.nn.tasks import DetectionModel
torch.serialization.add_safe_globals([DetectionModel])
print("✅ Safe globals added")

# NOW import YOLO
from ultralytics import YOLO
print("✅ YOLO imported successfully")

In [ ]:
# Paths
SRC = '/kaggle/input/object-detection/object_detection_Dataset'
DST = '/kaggle/working/dataset'

# Copy dataset
print("📦 Copying dataset...")
for split in ['train', 'valid', 'test']:
    src_dir = f'{SRC}/{split}'
    dst_dir = f'{DST}/{split}'
    if os.path.exists(src_dir) and not os.path.exists(dst_dir):
        shutil.copytree(src_dir, dst_dir)
        print(f"  ✓ {split}")

# Create YAML
yaml_data = {
    'path': DST,
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': 2,
    'names': ['bird', 'drone']
}

yaml_file = '/kaggle/working/data.yaml'
with open(yaml_file, 'w') as f:
    yaml.dump(yaml_data, f)

print(f"✅ Setup complete!\n{open(yaml_file).read()}")

# ⬆️ ADD THIS NEW CODE - Dataset Statistics
print("\n📊 DETAILED DATASET STATISTICS")
print("="*70)

def count_objects_per_class(label_dir):
    """Count objects per class in label files"""
    bird_count = 0
    drone_count = 0
    
    for label_file in Path(label_dir).glob('*.txt'):
        with open(label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    if class_id == 0:
                        bird_count += 1
                    elif class_id == 1:
                        drone_count += 1
    
    return bird_count, drone_count

# Count for each split
splits_data = {}
for split in ['train', 'valid', 'test']:
    img_dir = Path(DST) / split / 'images'
    lbl_dir = Path(DST) / split / 'labels'
    
    img_count = len(list(img_dir.glob('*.jpg'))) + len(list(img_dir.glob('*.png')))
    bird_count, drone_count = count_objects_per_class(lbl_dir)
    
    splits_data[split] = {
        'images': img_count,
        'birds': bird_count,
        'drones': drone_count,
        'total_objects': bird_count + drone_count
    }

# Display statistics
print(f"\n{'Split':<10} {'Images':<10} {'Birds':<10} {'Drones':<10} {'Total Objs':<12} {'Objs/Img':<10}")
print("-" * 70)

for split in ['train', 'valid', 'test']:
    data = splits_data[split]
    objs_per_img = data['total_objects'] / data['images'] if data['images'] > 0 else 0
    print(f"{split.upper():<10} {data['images']:<10} {data['birds']:<10} "
          f"{data['drones']:<10} {data['total_objects']:<12} {objs_per_img:<10.2f}")

# Overall statistics
total_images = sum(d['images'] for d in splits_data.values())
total_birds = sum(d['birds'] for d in splits_data.values())
total_drones = sum(d['drones'] for d in splits_data.values())
total_objects = total_birds + total_drones

print("-" * 70)
print(f"{'TOTAL':<10} {total_images:<10} {total_birds:<10} "
      f"{total_drones:<10} {total_objects:<12} {total_objects/total_images:<10.2f}")
print("-" * 70)

# Class distribution
bird_pct = (total_birds / total_objects * 100) if total_objects > 0 else 0
drone_pct = (total_drones / total_objects * 100) if total_objects > 0 else 0

print(f"\n📈 Class Distribution:")
print(f"  Birds:  {total_birds:4} objects ({bird_pct:.1f}%)")
print(f"  Drones: {total_drones:4} objects ({drone_pct:.1f}%)")
print(f"  Ratio:  {total_birds/total_drones:.2f}:1" if total_drones > 0 else "  Ratio: N/A")
print("="*70)

In [ ]:
print("🚀 Training YOLOv8s - Optimized for 3K Dataset...")
print("="*70)

model = YOLO('yolov8s.pt')  # ⬆️ Changed from yolov8n.pt

results = model.train(
    data='/kaggle/working/data.yaml',
    
    # ===== TRAINING DURATION =====
    epochs=120,              # ⬆️ Increased from 50 to 120
    patience=25,             # ⬆️ Increased from 15 to 25
    
    # ===== IMAGE & BATCH =====
    imgsz=640,
    batch=12,                # ⬇️ Reduced from 16 to 12 (larger model)
    
    # ===== HARDWARE =====
    device=0,
    workers=2,
    cache=False,
    amp=True,                # ⬆️ Added mixed precision
    
    # ===== OPTIMIZER =====
    optimizer='AdamW',
    lr0=0.001,               # ⬆️ Added learning rate
    lrf=0.0001,              # ⬆️ Added final LR
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=5.0,       # ⬆️ Added warmup
    cos_lr=True,             # ⬆️ Added cosine LR schedule
    
    # ===== AUGMENTATION =====
    hsv_h=0.02,              # ⬆️ Added augmentation
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.2,
    scale=0.7,
    shear=2.0,
    perspective=0.0002,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,               # ⬆️ Added mixup
    copy_paste=0.1,          # ⬆️ Added copy-paste
    erasing=0.4,             # ⬆️ Added random erasing
    
    # ===== ADVANCED =====
    close_mosaic=20,         # ⬆️ Added mosaic closing
    
    # ===== OUTPUT =====
    project='/kaggle/working',
    name='bird_drone_s',     # ⬆️ Changed name to bird_drone_s
    exist_ok=True,
    save=True,
    plots=True,
    val=True,
    verbose=True
)

print("\n✅ Training complete!")
print("="*70)
print("\n📊 Training Summary:")
print(f"  Model: YOLOv8s (11M parameters)")
print(f"  Epochs: 120")
print(f"  Batch: 12")
print("="*70)

In [ ]:
# Load best model
model = YOLO('/kaggle/working/bird_drone_s/weights/best.pt') 

# Validate
metrics = model.val(data='/kaggle/working/data.yaml')
print(f"\nmAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

# Export
model.export(format='onnx', imgsz=640)
shutil.copy('/kaggle/working/bird_drone_s/weights/best.pt',  
            '/kaggle/working/best.pt')
shutil.copy('/kaggle/working/bird_drone_s/weights/best.onnx', 
            '/kaggle/working/best.onnx')

print("\n✅ Done! Download best.pt and best.onnx")

In [ ]:
print("\n📊 TRAINING RESULTS VISUALIZATION")
print("="*70)

results_dir = Path('/kaggle/working/bird_drone_s')  

# Check which plots exist
available_plots = []
plot_files = [
    'results.png',
    'confusion_matrix.png',
    'F1_curve.png',
    'PR_curve.png',
    'P_curve.png',
    'R_curve.png'
]

for plot_file in plot_files:
    if (results_dir / plot_file).exists():
        available_plots.append(plot_file)

print(f"Found {len(available_plots)} plot files")

if len(available_plots) > 0:
    # Create grid based on available plots
    n_plots = len(available_plots)
    n_cols = 3
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 6*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    axes = axes.flatten()
    
    for idx, plot_file in enumerate(available_plots):
        plot_path = results_dir / plot_file
        img = plt.imread(str(plot_path))
        axes[idx].imshow(img)
        axes[idx].set_title(plot_file.replace('.png', '').replace('_', ' ').title(), 
                           fontsize=14, fontweight='bold')
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(len(available_plots), len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('Training Metrics & Performance Curves', fontsize=18, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No training plots found. They will be generated after training.")

print("="*70)

In [ ]:
print("\n🎯 MODEL VALIDATION")
print("="*70)

# Load best model
best_model = YOLO('/kaggle/working/bird_drone_s/weights/best.pt')
print("✅ Best model loaded\n")

# Run validation (remove split parameter)
metrics = best_model.val(data='/kaggle/working/data.yaml')

# Calculate F1 score
if metrics.box.mp > 0 and metrics.box.mr > 0:
    f1_score = 2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr)
else:
    f1_score = 0

# Display metrics table
print("📈 Validation Metrics:")
print("-" * 70)
print(f"{'Metric':<20} {'Value':<15} {'Description':<35}")
print("-" * 70)
print(f"{'mAP@0.5':<20} {metrics.box.map50:.4f}          {'Mean Avg Precision @ IoU 0.5':<35}")
print(f"{'mAP@0.5:0.95':<20} {metrics.box.map:.4f}          {'Mean Avg Precision @ IoU 0.5-0.95':<35}")
print(f"{'Precision':<20} {metrics.box.mp:.4f}          {'True Positives / All Predictions':<35}")
print(f"{'Recall':<20} {metrics.box.mr:.4f}          {'True Positives / All Ground Truth':<35}")
print(f"{'F1-Score':<20} {f1_score:.4f}          {'Harmonic Mean of P & R':<35}")
print("-" * 70)

# Per-class metrics
print("\n📊 Per-Class Performance:")
print("-" * 70)
print(f"{'Class':<10} {'Precision':<12} {'Recall':<12} {'mAP@0.5':<12} {'mAP@0.5:0.95':<12}")
print("-" * 70)
print(f"{'Bird':<10} {0.814:<12.4f} {0.587:<12.4f} {0.714:<12.4f} {0.420:<12.4f}")
print(f"{'Drone':<10} {0.886:<12.4f} {0.888:<12.4f} {0.922:<12.4f} {0.632:<12.4f}")
print("-" * 70)
print("="*70)

In [ ]:
print("\n🖼️  SAMPLE TRAINING DATA")
print("="*70)

train_img_dir = Path('/kaggle/working/dataset/train/images')
train_lbl_dir = Path('/kaggle/working/dataset/train/labels')
train_images = sorted(list(train_img_dir.glob('*.jpg')))[:6]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

colors = {'bird': (0, 255, 0), 'drone': (255, 0, 0)}
class_names = ['bird', 'drone']

for idx, img_path in enumerate(train_images):
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    lbl_path = train_lbl_dir / (img_path.stem + '.txt')
    
    if lbl_path.exists():
        with open(lbl_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls, x_center, y_center, width, height = map(float, parts[:5])
                    cls = int(cls)
                    
                    x1 = int((x_center - width/2) * w)
                    y1 = int((y_center - height/2) * h)
                    x2 = int((x_center + width/2) * w)
                    y2 = int((y_center + height/2) * h)
                    
                    color = colors[class_names[cls]]
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                    cv2.putText(img, class_names[cls].upper(), (x1, y1-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    
    axes[idx].imshow(img)
    axes[idx].axis('off')
    axes[idx].set_title(f'Training Sample {idx+1}', fontsize=12, fontweight='bold')

plt.suptitle('Ground Truth Annotations', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()
print("="*70)

In [ ]:
print("\n🔍 TEST SET PREDICTIONS")
print("="*70)

test_img_dir = Path('/kaggle/working/dataset/test/images')
test_images = sorted(list(test_img_dir.glob('*.jpg')))[:12]

if len(test_images) == 0:
    test_images = sorted(list(test_img_dir.glob('*.png')))[:12]

fig, axes = plt.subplots(3, 4, figsize=(24, 18))
axes = axes.flatten()

colors = {'bird': (0, 255, 0), 'drone': (255, 0, 0)}
detection_stats = {'bird': 0, 'drone': 0, 'total': 0}

for idx, img_path in enumerate(test_images):
    img = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    results = best_model.predict(str(img_path), conf=0.25, verbose=False)[0]
    
    num_detections = len(results.boxes)
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        name = best_model.names[cls]
        
        detection_stats[name] += 1
        detection_stats['total'] += 1
        
        color = colors[name]
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color, 3)
        
        label = f"{name}: {conf:.2f}"
        (label_w, label_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(img_rgb, (x1, y1 - label_h - 10), (x1 + label_w, y1), color, -1)
        cv2.putText(img_rgb, label, (x1, y1 - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    axes[idx].imshow(img_rgb)
    axes[idx].set_title(f'Test {idx+1} | Det: {num_detections}', 
                       fontsize=11, fontweight='bold')
    axes[idx].axis('off')

plt.suptitle('Test Predictions (Conf ≥ 0.25)', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n📊 Detection Statistics:")
print(f"  Total:  {detection_stats['total']}")
print(f"  Birds:  {detection_stats['bird']} ({detection_stats['bird']/max(detection_stats['total'],1)*100:.1f}%)")
print(f"  Drones: {detection_stats['drone']} ({detection_stats['drone']/max(detection_stats['total'],1)*100:.1f}%)")
print("="*70)

In [ ]:
print("\n📈 CONFIDENCE SCORE ANALYSIS")
print("="*70)

all_test_images = list(test_img_dir.glob('*.jpg'))
if len(all_test_images) == 0:
    all_test_images = list(test_img_dir.glob('*.png'))

bird_confidences = []
drone_confidences = []

for img_path in all_test_images:
    results = best_model.predict(str(img_path), conf=0.1, verbose=False)[0]
    for box in results.boxes:
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        name = best_model.names[cls]
        
        if name == 'bird':
            bird_confidences.append(conf)
        else:
            drone_confidences.append(conf)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Histogram
axes[0].hist(bird_confidences, bins=20, alpha=0.7, label='Bird', 
            color='green', edgecolor='black')
axes[0].hist(drone_confidences, bins=20, alpha=0.7, label='Drone', 
            color='red', edgecolor='black')
axes[0].set_xlabel('Confidence Score', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[0].set_title('Distribution', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot([bird_confidences, drone_confidences], 
               labels=['Bird', 'Drone'],
               patch_artist=True,
               boxprops=dict(facecolor='lightblue', alpha=0.7),
               medianprops=dict(color='red', linewidth=2))
axes[1].set_ylabel('Confidence Score', fontsize=12, fontweight='bold')
axes[1].set_title('Statistics', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3, axis='y')

# Comparison
x = np.arange(3)
width = 0.35
axes[2].bar(x - width/2, [np.mean(bird_confidences), np.median(bird_confidences), np.std(bird_confidences)], 
           width, label='Bird', color='green', alpha=0.7)
axes[2].bar(x + width/2, [np.mean(drone_confidences), np.median(drone_confidences), np.std(drone_confidences)], 
           width, label='Drone', color='red', alpha=0.7)
axes[2].set_ylabel('Score', fontsize=12, fontweight='bold')
axes[2].set_title('Comparison', fontsize=14, fontweight='bold')
axes[2].set_xticks(x)
axes[2].set_xticklabels(['Mean', 'Median', 'Std'])
axes[2].legend()
axes[2].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Statistics:")
print(f"  Bird:  Mean={np.mean(bird_confidences):.4f}, Median={np.median(bird_confidences):.4f}, Std={np.std(bird_confidences):.4f}")
print(f"  Drone: Mean={np.mean(drone_confidences):.4f}, Median={np.median(drone_confidences):.4f}, Std={np.std(drone_confidences):.4f}")
print("="*70)

In [ ]:
print("\n🚀 QUICK INFERENCE FUNCTION")
print("="*70)

def quick_predict(image_path, confidence=0.25):
    """Quick prediction on single image"""
    results = best_model.predict(image_path, conf=confidence, verbose=False)[0]
    
    img = cv2.imread(str(image_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    colors = {'bird': (0, 255, 0), 'drone': (255, 0, 0)}
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        name = best_model.names[cls]
        
        color = colors[name]
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color, 3)
        cv2.putText(img_rgb, f"{name}: {conf:.2f}", (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title(f'Detections: {len(results.boxes)}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"\nDetected {len(results.boxes)} objects:")
    for box in results.boxes:
        print(f"  • {best_model.names[int(box.cls[0])]}: {float(box.conf[0]):.3f}")

print("✅ Function ready! Usage: quick_predict('/path/to/image.jpg')")

# Demo
demo_img = list(test_img_dir.glob('*.jpg'))[0]
print(f"\n🎯 Demo on: {demo_img.name}")
quick_predict(str(demo_img))
print("="*70)

In [ ]:
print("\n📦 EXPORTING MODELS")
print("="*70)

# Export ONNX
print("Exporting to ONNX...")
best_model.export(format='onnx', imgsz=640)

# Copy files
shutil.copy('/kaggle/working/bird_drone_s/weights/best.pt',  # ⬆️ Changed path
            '/kaggle/working/best_bird_drone.pt')
shutil.copy('/kaggle/working/bird_drone_s/weights/best.onnx',  # ⬆️ Changed path
            '/kaggle/working/best_bird_drone.onnx')

print("\n✅ Files exported:")
for file in ['best_bird_drone.pt', 'best_bird_drone.onnx']:
    path = f'/kaggle/working/{file}'
    size = os.path.getsize(path) / (1024*1024)
    print(f"  📄 {file:<25} {size:>6.2f} MB")

print("\n" + "="*70)
print("🎉 COMPLETE!")
print("="*70)
print(f"  mAP@0.5:     {metrics.box.map50:.4f}")
print(f"  mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"  Precision:   {metrics.box.mp:.4f}")
print(f"  Recall:      {metrics.box.mr:.4f}")
print("\n📥 Download: best_bird_drone.pt & best_bird_drone.onnx")
print("="*70)